# INDIGO impact bond projects and organisations network plot
Network plot showing the relationship between projects and organisations for a subset of INDIGO projects.

In [ ]:
# Config
plot_title = 'Life Chances Fund - Project/Organisation Network'

plot_size = 2000

public_ids = [
    'INDIGO-POJ-0167',
    'INDIGO-POJ-0168',
    'INDIGO-POJ-0169',
    'INDIGO-POJ-0170',
    'INDIGO-POJ-0171',
    'INDIGO-POJ-0172',
    'INDIGO-POJ-0173',
    'INDIGO-POJ-0174',
    'INDIGO-POJ-0175',
    'INDIGO-POJ-0176',
    'INDIGO-POJ-0177',
    'INDIGO-POJ-0178',
    'INDIGO-POJ-0179',
    'INDIGO-POJ-0180',
    'INDIGO-POJ-0181',
    'INDIGO-POJ-0182',
    'INDIGO-POJ-0183',
    'INDIGO-POJ-0184',
    'INDIGO-POJ-0188',
    'INDIGO-POJ-0189',
    'INDIGO-POJ-0190',
    'INDIGO-POJ-0192',
    'INDIGO-POJ-0193',
    'INDIGO-POJ-0194',
    'INDIGO-POJ-0195',
    'INDIGO-POJ-0198',
    'INDIGO-POJ-0199',
    'INDIGO-POJ-0200',
    'INDIGO-POJ-0201',
]

project_annotation_include = [
    'INDIGO-POJ-0170',
    'INDIGO-POJ-0175',
    'INDIGO-POJ-0182',
    'INDIGO-POJ-0184',
    'INDIGO-POJ-0190',
    'INDIGO-POJ-0194',
    'INDIGO-POJ-0195',
    'INDIGO-POJ-0198',
    'INDIGO-POJ-0200', 
]

organisation_annotation_include = [
    'INDIGO-ORG-0008',
    'INDIGO-ORG-0121',
    'INDIGO-ORG-0171',
]

## INDIGO database API endpoint
Setup INDIGO database API endpoint and helper method for getting individual items from the API. This can be used with the project, fund, organisation and assessment_resource endpoints.

In [ ]:
import math
import requests
import textwrap

import networkx as nx
import numpy as np

import plotly.graph_objects as go


INDIGO_DATABASE_API = 'https://golab-indigo-data-store.herokuapp.com/app/api1'


def api_get_item(endpoint, public_id):
    """
    Get individual item details from the API

    E.g. 
    item = api_get_item('project', 'INDIGO-POJ-0158')
    """
    try:
        response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}/{public_id}')
        item = response.json()
        return item
    except Exception as e:
        print(f'\nFailed to retrieve {endpoint} "{public_id}".\nError: {e}')
        return False

## General helper methods
Setup helper methods for extracting values from a nested data dictionary and organisation role information from project data.

In [ ]:
ORGANISATION_ROLE_COMMISSIONER = 'Commissioner'
ORGANISATION_ROLE_INTERMEDIARY = 'Intermediary'
ORGANISATION_ROLE_INVESTOR = 'Investor'
ORGANISATION_ROLE_PROVIDER = 'Provider'


def extract_value(data, keys, default):
    """
    Safe method to get value from nested dictionary with default value fallback
    """
    try:
        result = data
        for key in keys:
            if result == default:
                break
            result = result.get(key) or default
        return result

    except Exception as e:
        print(f'Error: {e}')
        return default


def add_to_organisation_roles(organisation_roles, role, data):
      """
      Add/update the organisation_role
      """
      id = extract_value(data, ['organisation_id', 'value'], '')
      if id:
          try:
              organisation_roles[id].add(role)
          except KeyError:
              organisation_roles[id] = {role}

      return organisation_roles


def get_organisation_roles(project_data):
    """
    Returns an organisations roles dictionary for a project
    that contains a set of roles for each organisation.

    e.g. {
        'INDIGO-ORG-0001': {'Commissioner'},
        'INDIGO-ORG-0002': {'Investor'},
        'INDIGO-ORG-0003': {'Provider'},
        'INDIGO-ORG-0004': {'Intermediary'},
    }
    """
    organisation_roles = {}

    for commitment in project_data.get('outcome_payment_commitments', []):
        organisation_roles = add_to_organisation_roles(
            organisation_roles,
            ORGANISATION_ROLE_COMMISSIONER,
            commitment,
        )

    for investment in project_data.get('investments', []):
        organisation_roles = add_to_organisation_roles(
            organisation_roles,
            ORGANISATION_ROLE_INVESTOR,
            investment,
        )

    for provision in project_data.get('service_provisions', []):
        organisation_roles = add_to_organisation_roles(
            organisation_roles,
            ORGANISATION_ROLE_PROVIDER,
            provision,
        )

    for intermediary in project_data.get('intermediary_services', []):
        organisation_roles = add_to_organisation_roles(
            organisation_roles,
            ORGANISATION_ROLE_INTERMEDIARY,
            intermediary,
        )

    return organisation_roles


def update_organisation_roles(project_data, organisation_roles):
    """
    Update an organisations roles dictionary for a project
    """
    for k, v in get_organisation_roles(project_data).items():
        try:
            organisation_roles[k].update(v)
        except KeyError:
            organisation_roles[k] = v

    return organisation_roles

# Plot helper methods
Helper methods for building nodes and edges of the network plot.

In [ ]:
ANNOTATION_FONT_SIZE = 13
WRAPPER = textwrap.TextWrapper(width=40)

def make_line_trace(edge_x, edge_y, line_width=1):
    """
    Draw lines between nodes
    """
    return go.Scatter(
        x=edge_x,
        y=edge_y,
        mode='lines',
        name='',
        line=dict(
            color=ORGANISATION_COLOUR_LINE,
            width=line_width,
        ),
        hoverinfo='none',
        showlegend=False,
    )


def make_scatter_trace(data):
    """
    Draw nodes scatter plot
    """
    return go.Scatter(
        x=data['x'],
        y=data['y'],
        mode='markers',
        name=data['name'],
        marker=dict(
            symbol='circle-dot',
            size=data['sizes'],
            color=data['colour'],
            line=dict(color=data['line_colour'], width=2)
        ),
        text=data['labels'],
        hoverinfo='text',
    )


def make_scatter_dict(name, colour, line_colour):
    """
    Make an organisation dict
    """
    return {
        'name': name,
        'x': [],
        'y': [],
        'sizes': [],
        'labels': [],
        'colour': colour,
        'line_colour': line_colour,
    }


def update_org(org, x, y, size, label):
    """
    Update and organisation dict
    """
    org['x'].append(x)
    org['y'].append(y)
    org['sizes'].append(size)
    org['labels'].append(label)


def get_edges_positions(pos, edges):
    """
    Generate lists of x, y edge positions from layout position data
    """
    edge_x = []
    edge_y = []

    for edge in edges:
        source = edge[0]
        target = edge[1]

        sx, sy = pos[source]
        tx, ty = pos[target]

        edge_x += [sx, tx, None]
        edge_y += [sy, ty, None]

    return edge_x, edge_y


def add_annotation(fig, title, values, colour='black', bgcolor='white'):
    """
    Adds an annotation label to the plot
    """
    x = values['x']
    y = values['y']
    size = values['size']

    fig.add_annotation(
        x=x, 
        y=y,
        text=title,
        showarrow=True,
        font_color='#333',
        font_size=ANNOTATION_FONT_SIZE,
        yanchor='bottom',
        arrowside='end',
        arrowwidth=2,
        arrowhead=2,
        yshift=size / 2,
        bgcolor=bgcolor,
        standoff=0,
        bordercolor=colour,
        borderwidth=2,
        borderpad=8,
        ax=0,
        ay=-50,
    )

## Get project data
Call the INDIGO API 'project' endpoint and retrieve the data used for the plot.

In [ ]:
data = {}
endpoint = 'project'

for public_id in public_ids:
    data[public_id] = api_get_item(endpoint, public_id)

## Generate plot data
Generate the network graph data and positions.

In [ ]:
edges = set()
project_nodes = []
organisation_nodes = []
organisation_roles = {}
seen = []

# Generate node and edge graph data
for public_id in public_ids:
    
    project_data = data[public_id]['project']['data']

    organisations = project_data['organisations']

    if not organisations:
        continue

    #project_id = public_id
    project_node = (public_id, {'type': 'project', 'title': project_data['name']['value']})
    project_nodes.append(project_node)

    organisation_roles = update_organisation_roles(project_data, organisation_roles)

    orgs = [{'public_id': org['id'], 'title': org['name']['value']} for org in organisations]

    # May need to inject other params in here for org type
    organisation_nodes += [
        (
            item['public_id'],
            {'type': 'org', 'title': item['title']}
        )
        for item in orgs if not(item['public_id'] in seen or seen.append(item['public_id']))
    ]

    combinations = [(public_id, item['public_id']) for item in orgs]
    edges.update(combinations)

print('Project nodes:', len(project_nodes))
print('Organisation nodes:', len(organisation_nodes))
print('Edges:', len(edges))

# Sort the data so we get repeatable graph results across platforms
project_nodes.sort(key=lambda x: x[0])
organisation_nodes.sort(key=lambda x: x[0])

nodes = project_nodes + organisation_nodes

#Create a network graph from nodes and edges
g = nx.Graph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)

# Generate node positions
pos = nx.spring_layout(g, dim=2, seed=1, iterations=1000)

## Build the plot


In [ ]:
LINE_COLOUR = '#cccccc'

PROJECT_COLOUR = '#3c77ab'
PROJECT_COLOUR_LINE = '#ffffff'

ORGANISATION_COLOUR = '#ffffff'
ORGANISATION_COLOUR_LINE = LINE_COLOUR

ORGANISATION_COLOUR_ROLE = '#a05195'

ORGANISATION_COMMISSIONER_COLOUR = '#d45087'
ORGANISATION_INTERMEDIARY_COLOUR = '#27ACCE'
ORGANISATION_INVESTOR_COLOUR = '#007c43'
ORGANISATION_PROVIDER_COLOUR = '#ffa600'

project = make_scatter_dict('Project', PROJECT_COLOUR, PROJECT_COLOUR_LINE)

project_annotations = {}
organisation_annotations = {}

for node in project_nodes:
    public_id = node[0]
    title = node[1]['title']
    x, y = pos[public_id]
    size = int(15 * math.log(g.degree(public_id) + 1))
    num_orgs = g.degree(public_id)

    project['x'].append(x)
    project['y'].append(y)
    project['sizes'].append(size)

    project['labels'].append((
        f'<b>Title:</b> {title}<br>'
        f'<b>Project ID:</b> {public_id}<br>'
        f'<b>Associated organisations:</b> {num_orgs}'
    ))

    project_annotations[public_id] = {
        'title': title,
        'x': x,
        'y': y,
        'size': size,
        'num_orgs': num_orgs,
    }

# Build organisation nodes
commissioner = make_scatter_dict(ORGANISATION_ROLE_COMMISSIONER, ORGANISATION_COLOUR, ORGANISATION_COMMISSIONER_COLOUR)
investor = make_scatter_dict(ORGANISATION_ROLE_INVESTOR, ORGANISATION_COLOUR, ORGANISATION_INVESTOR_COLOUR)
provider = make_scatter_dict(ORGANISATION_ROLE_PROVIDER, ORGANISATION_COLOUR, ORGANISATION_PROVIDER_COLOUR)
intermediary = make_scatter_dict(ORGANISATION_ROLE_INTERMEDIARY, ORGANISATION_COLOUR, ORGANISATION_INTERMEDIARY_COLOUR)
no_role = make_scatter_dict('No role', ORGANISATION_COLOUR, ORGANISATION_COLOUR_LINE)

for node in organisation_nodes:
    public_id = node[0]

    title = node[1]['title']
    x, y = pos[public_id]
    size = int(15 * math.log(g.degree(public_id) + 1))
    associated_projects = g.degree(public_id)

    roles = list(organisation_roles.get(public_id, []))
    roles_str = ', '.join(roles)

    label = (
        f'<b>Title:</b> {title}<br>'
        f'<b>Organisation ID:</b> {public_id}<br>'
        f'<b>Associated projects:</b> {associated_projects}<br>'
        f'<b>Role:</b> {roles_str}<br>'
    )

    if ORGANISATION_ROLE_COMMISSIONER in roles:
        update_org(commissioner, x, y, size, label)

    if ORGANISATION_ROLE_INVESTOR in roles:
        update_org(investor, x, y, size, label)

    if ORGANISATION_ROLE_PROVIDER in roles:
        update_org(provider, x, y, size, label)

    if ORGANISATION_ROLE_INTERMEDIARY in roles:
        update_org(intermediary, x, y, size, label)

    if (
        ORGANISATION_ROLE_COMMISSIONER not in roles and
        ORGANISATION_ROLE_INVESTOR not in roles and
        ORGANISATION_ROLE_PROVIDER not in roles and
        ORGANISATION_ROLE_INTERMEDIARY not in roles
    ):
        update_org(no_role, x, y, size, label)
    
    organisation_annotations[public_id] = {
        'title': title,
        'x': x,
        'y': y,
        'size': size,
        'roles': roles,
        'associated_projects': associated_projects,
    }

# Assign edges
edge_x, edge_y = get_edges_positions(pos, edges)

# Draw plots
trace_lines = make_line_trace(edge_x, edge_y)
trace_projects = make_scatter_trace(project)
trace_commissioners = make_scatter_trace(commissioner)
trace_investors = make_scatter_trace(investor)
trace_providers = make_scatter_trace(provider)
trace_intermediaries = make_scatter_trace(intermediary)
trace_no_role = make_scatter_trace(no_role)

plot_data = [
    trace_lines,
    trace_projects,
    trace_commissioners,
    trace_investors,
    trace_providers,
    trace_intermediaries,
    trace_no_role,
]

fig = go.Figure(data=plot_data)

# Set initial axis range base on min/max of data
axis_data = np.vstack(list(pos.values()))
xaxis_range = [np.min(axis_data[:, 0]) - 0.1, np.max(axis_data[:, 0]) + 0.1]
yaxis_range = [np.min(axis_data[:, 1]) - 0.1, np.max(axis_data[:, 1]) + 0.1]

fig.update_layout(
    title=plot_title,
    title_x=0.5,
    title_font_size=24,
    height=plot_size,
    width=plot_size,
    xaxis_range=xaxis_range,
    yaxis_range=yaxis_range,
    legend_orientation='h',
    legend_x=0.5,
    legend_y=0,
    legend_xanchor='center',
    legend_itemsizing='constant',
    legend_font_size=17,
    margin={'t': 40, 'b': 0, 'l': 0, 'r': 0},
    xaxis_visible=False,
    yaxis_visible=False,
    paper_bgcolor='#fff',
    plot_bgcolor='#fff',
)

# Add selected project annotations
for public_id, values in project_annotations.items():
    if public_id in project_annotation_include:

        title = '<br>'.join(WRAPPER.wrap(values['title']))

        num_orgs = values['num_orgs']        
        extra = '<br>'.join(WRAPPER.wrap(f'({num_orgs} organisations)'))

        title = (
            f'{public_id}<br>'
            f'<b>{title}</b><br>'
            f'<i>{extra}</i>'
        )

        add_annotation(fig, title, values, PROJECT_COLOUR, 'rgba(240, 240, 255, 0.85)')

# Add selected organisation annotations
for public_id, values in organisation_annotations.items():
    if public_id in organisation_annotation_include:

        title = '<br>'.join(WRAPPER.wrap(values['title']))

        roles = values['roles']
        roles_str = ', '.join(roles)
                  
        associated_projects = values['associated_projects']

        extra = '<br>'.join(WRAPPER.wrap(f'({roles_str}, {associated_projects} projects)'))

        title = (
            f'{public_id}<br>'
            f'<b>{title}</b><br>'
            f'<i>{extra}</i>'
        )

        colour = ORGANISATION_COLOUR_ROLE
        bgcolor = 'rgba(255, 255, 255, 0.85)'

        if 'Investor' in roles:
            colour = ORGANISATION_INVESTOR_COLOUR
        
        if 'Intermediary' in roles:
            colour = ORGANISATION_INTERMEDIARY_COLOUR

        if 'Commissioner' in roles:
            colour = ORGANISATION_COMMISSIONER_COLOUR

        if 'Provider' in roles:
            colour = ORGANISATION_PROVIDER_COLOUR

        add_annotation(fig, title, values, colour, bgcolor)
        

fig.show()

## Important Notice and Disclaimer on INDIGO Data
<sub><sup>
INDIGO data are shared for research and policy analysis purposes. INDIGO data can be used to support a range of insights, for example, to understand the social outcomes that projects aim to improve, the network of organisations across projects, trends, scales, timelines and summary information. The collaborative system by which we collect, process, and share data is designed to advance data-sharing norms, harmonise data definitions and improve data use. These data are NOT shared for auditing, investment, or legal purposes. Please independently verify any data that you might use in decision making. We provide no guarantees or assurances as to the quality of these data. Data may be inaccurate, incomplete, inconsistent, and/or not current for various reasons: INDIGO is a collaborative and iterative initiative that mostly relies on projects all over the world volunteering to share their data. We have a system for processing information and try to attribute data to named sources, but we do not audit, cross-check, or verify all information provided to us. It takes time and resources to share data, which may not have been included in a project’s budget. Many of the projects are ongoing and timely updates may not be available. Different people may have different interpretations of data items and definitions. Even when data are high quality, interpretation or generalisation to different contexts may not be possible and/or requires additional information and/or expertise. Help us improve our data quality: email us at indigo@bsg.ox.ac.uk if you have data on new projects, changes or performance updates on current projects, clarifications or corrections on our data, and/or confidentiality or sensitivity notices. Please also give input via the INDIGO Data Definitions Improvement Tool and INDIGO Feedback Questionnaire.
</sup></sub>